In [2]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [3]:
data = pd.read_csv("archive/train.txt", sep=';')
data.columns = ["Text", "Emotions"]

In [4]:
data.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [8]:
data.describe()

,Text,Emotions
count,15999,15999
unique,15968,6
top,i cant escape the tears of sadness and just tr...,joy
freq,2,5362


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15999 entries, 0 to 15998
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      15999 non-null  object
 1   Emotions  15999 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


In [10]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [12]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [13]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [14]:
one_hot_labels = keras.utils.to_categorical(labels)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2)

In [22]:
model = Sequential()

model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_length))

model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(one_hot_labels[0]), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [23]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/10
400/400 [==============================] - 10s 24ms/step - loss: 1.3387 - accuracy: 0.4927 - val_loss: 0.8132 - val_accuracy: 0.7422
Epoch 2/10
400/400 [==============================] - 10s 24ms/step - loss: 0.3200 - accuracy: 0.9016 - val_loss: 0.5303 - val_accuracy: 0.8256
Epoch 3/10
400/400 [==============================] - 10s 26ms/step - loss: 0.0543 - accuracy: 0.9853 - val_loss: 0.5826 - val_accuracy: 0.8256
Epoch 4/10
400/400 [==============================] - 10s 25ms/step - loss: 0.0278 - accuracy: 0.9942 - val_loss: 0.6133 - val_accuracy: 0.8306
Epoch 5/10
400/400 [==============================] - 10s 25ms/step - loss: 0.0177 - accuracy: 0.9953 - val_loss: 0.6515 - val_accuracy: 0.8225
Epoch 6/10
400/400 [==============================] - 10s 25ms/step - loss: 0.0171 - accuracy: 0.9965 - val_loss: 0.6434 - val_accuracy: 0.8288
Epoch 7/10
400/400 [==============================] - 10s 24ms/step - loss: 0.0137 - accuracy: 0.9964 - val_loss: 0.6947 - val_accuracy:

In [24]:
input_text = "She didn't come today because she lost her dog yesterday!"

input_sequence = tokenizer.texts_to_sequences([input_text])
padded_inpt_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_inpt_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])

1/1 [==============================] - 0s 64ms/step


In [25]:
predicted_label

array(['sadness'], dtype='<U8')